# Vexpresso Walkthrough

## In this walkthrough, we will be using a small dataset of Pokemon statistics to showcase vexpresso's capabilities

### Imports

In [1]:
import json
import pandas as pd
from langchain.embeddings import HuggingFaceHubEmbeddings, HuggingFaceEmbeddings
import numpy as np
import daft
import vexpresso
from vexpresso.retriever import NumpyRetriever


/home/shyam/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create Collection
### This may take some time because we are embedding 809 entries

In [2]:
with open("./data/pokedex.json", 'r') as f:
    stuff = json.load(f)

df = pd.DataFrame(stuff)

### Pokemon data

In [3]:
df.head(10)

,id,name,type,base,species,description,evolution,profile,image
0,1,"{'english': 'Bulbasaur', 'japanese': 'フシギダネ', ...","[Grass, Poison]","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. A...",Seed Pokémon,Bulbasaur can be seen napping in bright sunlig...,"{'next': [['2', 'Level 16']]}","{'height': '0.7 m', 'weight': '6.9 kg', 'egg':...",{'sprite': 'https://raw.githubusercontent.com/...
1,2,"{'english': 'Ivysaur', 'japanese': 'フシギソウ', 'c...","[Grass, Poison]","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. A...",Seed Pokémon,There is a bud on this Pokémon’s back. To supp...,"{'prev': ['1', 'Level 16'], 'next': [['3', 'Le...","{'height': '1 m', 'weight': '13 kg', 'egg': ['...",{'sprite': 'https://raw.githubusercontent.com/...
2,3,"{'english': 'Venusaur', 'japanese': 'フシギバナ', '...","[Grass, Poison]","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. A...",Seed Pokémon,There is a large flower on Venusaur’s back. Th...,"{'prev': ['2', 'Level 32']}","{'height': '2 m', 'weight': '100 kg', 'egg': [...",{'sprite': 'https://raw.githubusercontent.com/...
3,4,"{'english': 'Charmander', 'japanese': 'ヒトカゲ', ...",[Fire],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. A...",Lizard Pokémon,The flame that burns at the tip of its tail is...,"{'next': [['5', 'Level 16']]}","{'height': '0.6 m', 'weight': '8.5 kg', 'egg':...",{'sprite': 'https://raw.githubusercontent.com/...
4,5,"{'english': 'Charmeleon', 'japanese': 'リザード', ...",[Fire],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. A...",Flame Pokémon,Charmeleon mercilessly destroys its foes using...,"{'prev': ['4', 'Level 16'], 'next': [['6', 'Le...","{'height': '1.1 m', 'weight': '19 kg', 'egg': ...",{'sprite': 'https://raw.githubusercontent.com/...
5,6,"{'english': 'Charizard', 'japanese': 'リザードン', ...","[Fire, Flying]","{'HP': 78, 'Attack': 84, 'Defense': 78, 'Sp. A...",Flame Pokémon,Charizard flies around the sky in search of po...,"{'prev': ['5', 'Level 36']}","{'height': '1.7 m', 'weight': '90.5 kg', 'egg'...",{'sprite': 'https://raw.githubusercontent.com/...
6,7,"{'english': 'Squirtle', 'japanese': 'ゼニガメ', 'c...",[Water],"{'HP': 44, 'Attack': 48, 'Defense': 65, 'Sp. A...",Tiny Turtle Pokémon,Squirtle’s shell is not merely used for protec...,"{'next': [['8', 'Level 16']]}","{'height': '0.5 m', 'weight': '9 kg', 'egg': [...",{'sprite': 'https://raw.githubusercontent.com/...
7,8,"{'english': 'Wartortle', 'japanese': 'カメール', '...",[Water],"{'HP': 59, 'Attack': 63, 'Defense': 80, 'Sp. A...",Turtle Pokémon,"Its tail is large and covered with a rich, thi...","{'prev': ['7', 'Level 16'], 'next': [['9', 'Le...","{'height': '1 m', 'weight': '22.5 kg', 'egg': ...",{'sprite': 'https://raw.githubusercontent.com/...
8,9,"{'english': 'Blastoise', 'japanese': 'カメックス', ...",[Water],"{'HP': 79, 'Attack': 83, 'Defense': 100, 'Sp. ...",Shellfish Pokémon,Blastoise has water spouts that protrude from ...,"{'prev': ['8', 'Level 36']}","{'height': '1.6 m', 'weight': '85.5 kg', 'egg'...",{'sprite': 'https://raw.githubusercontent.com/...
9,10,"{'english': 'Caterpie', 'japanese': 'キャタピー', '...",[Bug],"{'HP': 45, 'Attack': 30, 'Defense': 35, 'Sp. A...",Worm Pokémon,"Its body is soft and weak. In nature, its perp...","{'next': [['11', 'Level 7']]}","{'height': '0.3 m', 'weight': '2.9 kg', 'egg':...",{'sprite': 'https://raw.githubusercontent.com/...


In [4]:
# embeddings = HuggingFaceEmbeddings()
# def embedding_fn(content):
#     return np.array(embeddings.embed_documents(content))

collection = vexpresso.create(
    content="description",
    metadata="./data/pokedex.json",
    embedding_fn=HuggingFaceEmbeddings(),
    retriever=NumpyRetriever()
)

2023-06-01 20:06:07.818 | INFO     | daft.context:runner:88 - Using PyRunner


#### Or load with df

```python
with open("./data/pokedex.json", 'r') as f:
    stuff = json.load(f)

df = pd.DataFrame(stuff)

collection = vexpresso.create(
    content="description",
    metadata=df,
    embedding_fn=HuggingFaceEmbeddings(),
    retriever=NumpyRetriever()
)
```

In [5]:
collection.show(5)

descriptionUtf8,idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",embeddings_descriptionPython,modal_map_indexInt64
Bulbasaur can be seen napping in bright sunlight. There i...,1,"{'chinese': '妙蛙种子', 'english': 'Bulbasaur', 'french': 'Bu...","['Grass', 'Poison']","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. Attack': 65,...",Seed Pokémon,"{'next': [['2', 'Level 16']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",0
There is a bud on this Pokémon’s back. To support its wei...,2,"{'chinese': '妙蛙草', 'english': 'Ivysaur', 'french': 'Herbi...","['Grass', 'Poison']","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. Attack': 80,...",Seed Pokémon,"{'next': [['3', 'Level 32']], 'prev': ['1', 'Level 16']}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",1
There is a large flower on Venusaur’s back. The flower is...,3,"{'chinese': '妙蛙花', 'english': 'Venusaur', 'french': 'Flor...","['Grass', 'Poison']","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. Attack': 100...",Seed Pokémon,"{'next': None, 'prev': ['2', 'Level 32']}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",2
The flame that burns at the tip of its tail is an indicat...,4,"{'chinese': '小火龙', 'english': 'Charmander', 'french': 'Sa...",['Fire'],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. Attack': 60,...",Lizard Pokémon,"{'next': [['5', 'Level 16']], 'prev': None}","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",3
Charmeleon mercilessly destroys its foes using its sharp ...,5,"{'chinese': '火恐龙', 'english': 'Charmeleon', 'french': 'Re...",['Fire'],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. Attack': 80,...",Flame Pokémon,"{'next': [['6', 'Level 36']], 'prev': ['4', 'Level 16']}","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",4


## Querying the collection's embeddings, lazy execution
### This doesn't run anything until `execute()` is called

In [6]:
sleepy_pokemon = collection.query(query={"description":["Loves to sleep"]}, k=10)
sleepy_pokemon.show(5)

descriptionUtf8,idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",embeddings_descriptionPython,modal_map_indexInt64
Bulbasaur can be seen napping in bright sunlight. There i...,1,"{'chinese': '妙蛙种子', 'english': 'Bulbasaur', 'french': 'Bu...","['Grass', 'Poison']","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. Attack': 65,...",Seed Pokémon,"{'next': [['2', 'Level 16']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",0
There is a bud on this Pokémon’s back. To support its wei...,2,"{'chinese': '妙蛙草', 'english': 'Ivysaur', 'french': 'Herbi...","['Grass', 'Poison']","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. Attack': 80,...",Seed Pokémon,"{'next': [['3', 'Level 32']], 'prev': ['1', 'Level 16']}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",1
There is a large flower on Venusaur’s back. The flower is...,3,"{'chinese': '妙蛙花', 'english': 'Venusaur', 'french': 'Flor...","['Grass', 'Poison']","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. Attack': 100...",Seed Pokémon,"{'next': None, 'prev': ['2', 'Level 32']}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",2
The flame that burns at the tip of its tail is an indicat...,4,"{'chinese': '小火龙', 'english': 'Charmander', 'french': 'Sa...",['Fire'],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. Attack': 60,...",Lizard Pokémon,"{'next': [['5', 'Level 16']], 'prev': None}","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",3
Charmeleon mercilessly destroys its foes using its sharp ...,5,"{'chinese': '火恐龙', 'english': 'Charmeleon', 'french': 'Re...",['Fire'],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. Attack': 80,...",Flame Pokémon,"{'next': [['6', 'Level 36']], 'prev': ['4', 'Level 16']}","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",4


In [7]:
sleepy_pokemon.execute().show(5)

descriptionUtf8,idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",embeddings_descriptionPython,modal_map_indexInt64,scoreFloat64
It has no interest in anything other than eating. Even if...,143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'french': 'Ronfl...",['Normal'],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp. Attack': 6...",Sleeping Pokémon,"{'next': None, 'prev': ['446', 'high Friendship']}","{'ability': [['Immunity', 'false'], ['Thick Fat', 'false'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",142,0.561686
"While sleeping, it generates electricity in the sacs in i...",25,"{'chinese': '皮卡丘', 'english': 'Pikachu', 'french': 'Pikac...",['Electric'],"{'HP': 35, 'Attack': 55, 'Defense': 40, 'Sp. Attack': 50,...",Mouse Pokémon,"{'next': [['26', 'use Thunder Stone']], 'prev': ['172', '...","{'ability': [['Static', 'false'], ['Lightning Rod', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",24,0.474437
"It builds nests that double as traps, lying in wait for i...",328,"{'chinese': '大颚蚁', 'english': 'Trapinch', 'french': 'Krak...",['Ground'],"{'HP': 45, 'Attack': 100, 'Defense': 45, 'Sp. Attack': 45...",Ant Pit Pokémon,"{'next': [['329', 'Level 35']], 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Arena Trap', 'f...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",327,0.450725
"It feels relaxed in tight, dark places and has been known...",722,"{'chinese': '木木枭', 'english': 'Rowlet', 'french': 'Brindi...","['Grass', 'Flying']","{'HP': 68, 'Attack': 55, 'Defense': 55, 'Sp. Attack': 50,...",Grass Quill Pokémon,"{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Reach', 'true'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",721,0.445208
"When it sleeps, it pulls its limbs into its body and its ...",554,"{'chinese': '火红不倒翁', 'english': 'Darumaka', 'french': 'Da...",['Fire'],"{'HP': 70, 'Attack': 90, 'Defense': 45, 'Sp. Attack': 15,...",Zen Charm Pokémon,"{'next': [['555', 'use Ice Stone']], 'prev': None}","{'ability': [['Hustle', 'false'], ['Inner Focus', 'true']...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",553,0.430829


### This allows for query chaining

In [8]:
sleepy_water_pokemon = collection.query(query={"description":["Loves to sleep"]}, k=10).query(query={"description":["Water body"]}, k=5).execute()

#### Show daft DataFrame

In [9]:
sleepy_water_pokemon.show(5)

descriptionUtf8,idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",embeddings_descriptionPython,modal_map_indexInt64,scoreFloat64
"When it sleeps, it pulls its limbs into its body and its ...",554,"{'chinese': '火红不倒翁', 'english': 'Darumaka', 'french': 'Da...",['Fire'],"{'HP': 70, 'Attack': 90, 'Defense': 45, 'Sp. Attack': 15,...",Zen Charm Pokémon,"{'next': [['555', 'use Ice Stone']], 'prev': None}","{'ability': [['Hustle', 'false'], ['Inner Focus', 'true']...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",553,0.172056
"It builds nests that double as traps, lying in wait for i...",328,"{'chinese': '大颚蚁', 'english': 'Trapinch', 'french': 'Krak...",['Ground'],"{'HP': 45, 'Attack': 100, 'Defense': 45, 'Sp. Attack': 45...",Ant Pit Pokémon,"{'next': [['329', 'Level 35']], 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Arena Trap', 'f...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",327,0.14965
"Although it’s tough, it can’t handle cold well. When nigh...",127,"{'chinese': '凯罗斯', 'english': 'Pinsir', 'french': 'Scarab...",['Bug'],"{'HP': 65, 'Attack': 125, 'Defense': 100, 'Sp. Attack': 5...",Stag Beetle Pokémon,"{'next': None, 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Mold Breaker', ...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",126,0.128657
Its body is always warm. Trainers who live in cold areas ...,661,"{'chinese': '小箭雀', 'english': 'Fletchling', 'french': 'Pa...","['Normal', 'Flying']","{'HP': 45, 'Attack': 50, 'Defense': 43, 'Sp. Attack': 40,...",Tiny Robin Pokémon,"{'next': [['662', 'Level 17']], 'prev': None}","{'ability': [['Big Pecks', 'false'], ['Gale Wings', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",660,0.111246
"It feels relaxed in tight, dark places and has been known...",722,"{'chinese': '木木枭', 'english': 'Rowlet', 'french': 'Brindi...","['Grass', 'Flying']","{'HP': 68, 'Attack': 55, 'Defense': 55, 'Sp. Attack': 50,...",Grass Quill Pokémon,"{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Reach', 'true'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",721,0.111175


#### Show pandas

In [10]:
sleepy_water_pokemon.to_pandas()

,description,id,name,type,base,species,evolution,profile,image,embeddings_description,modal_map_index,score
0,"When it sleeps, it pulls its limbs into its bo...",554,"{'chinese': '火红不倒翁', 'english': 'Darumaka', 'f...",[Fire],"{'HP': 70, 'Attack': 90, 'Defense': 45, 'Sp. A...",Zen Charm Pokémon,"{'next': [['555', 'use Ice Stone']], 'prev': N...","{'ability': [['Hustle', 'false'], ['Inner Focu...",{'hires': 'https://raw.githubusercontent.com/P...,"[-0.006264317315071821, -0.06521143019199371, ...",553,0.172056
1,"It builds nests that double as traps, lying in...",328,"{'chinese': '大颚蚁', 'english': 'Trapinch', 'fre...",[Ground],"{'HP': 45, 'Attack': 100, 'Defense': 45, 'Sp. ...",Ant Pit Pokémon,"{'next': [['329', 'Level 35']], 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Aren...",{'hires': 'https://raw.githubusercontent.com/P...,"[0.00905408151447773, -0.04088602587580681, -0...",327,0.149650
2,"Although it’s tough, it can’t handle cold well...",127,"{'chinese': '凯罗斯', 'english': 'Pinsir', 'frenc...",[Bug],"{'HP': 65, 'Attack': 125, 'Defense': 100, 'Sp....",Stag Beetle Pokémon,"{'next': None, 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Mold...",{'hires': 'https://raw.githubusercontent.com/P...,"[0.01692618988454342, -0.03155365213751793, -0...",126,0.128657
3,Its body is always warm. Trainers who live in ...,661,"{'chinese': '小箭雀', 'english': 'Fletchling', 'f...","[Normal, Flying]","{'HP': 45, 'Attack': 50, 'Defense': 43, 'Sp. A...",Tiny Robin Pokémon,"{'next': [['662', 'Level 17']], 'prev': None}","{'ability': [['Big Pecks', 'false'], ['Gale Wi...",{'hires': 'https://raw.githubusercontent.com/P...,"[-0.0028398018330335617, -0.06399963796138763,...",660,0.111246
4,"It feels relaxed in tight, dark places and has...",722,"{'chinese': '木木枭', 'english': 'Rowlet', 'frenc...","[Grass, Flying]","{'HP': 68, 'Attack': 55, 'Defense': 55, 'Sp. A...",Grass Quill Pokémon,"{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Rea...",{'hires': 'https://raw.githubusercontent.com/P...,"[0.003820237936452031, -0.06739205121994019, -...",721,0.111175


## Querying the collection's embeddings, eager execution
### This is useful if you don't need to chain calls together and just want to get the results back immediately

In [11]:
query = "Loves to sleep"
sleepy_pokemon = collection.query(query={"description":["Loves to sleep"]}, k=10, lazy=False)
sleepy_pokemon.show(5)

descriptionUtf8,idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",embeddings_descriptionPython,modal_map_indexInt64,scoreFloat64
It has no interest in anything other than eating. Even if...,143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'french': 'Ronfl...",['Normal'],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp. Attack': 6...",Sleeping Pokémon,"{'next': None, 'prev': ['446', 'high Friendship']}","{'ability': [['Immunity', 'false'], ['Thick Fat', 'false'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",142,0.561686
"While sleeping, it generates electricity in the sacs in i...",25,"{'chinese': '皮卡丘', 'english': 'Pikachu', 'french': 'Pikac...",['Electric'],"{'HP': 35, 'Attack': 55, 'Defense': 40, 'Sp. Attack': 50,...",Mouse Pokémon,"{'next': [['26', 'use Thunder Stone']], 'prev': ['172', '...","{'ability': [['Static', 'false'], ['Lightning Rod', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",24,0.474437
"It builds nests that double as traps, lying in wait for i...",328,"{'chinese': '大颚蚁', 'english': 'Trapinch', 'french': 'Krak...",['Ground'],"{'HP': 45, 'Attack': 100, 'Defense': 45, 'Sp. Attack': 45...",Ant Pit Pokémon,"{'next': [['329', 'Level 35']], 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Arena Trap', 'f...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",327,0.450725
"It feels relaxed in tight, dark places and has been known...",722,"{'chinese': '木木枭', 'english': 'Rowlet', 'french': 'Brindi...","['Grass', 'Flying']","{'HP': 68, 'Attack': 55, 'Defense': 55, 'Sp. Attack': 50,...",Grass Quill Pokémon,"{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Reach', 'true'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",721,0.445208
"When it sleeps, it pulls its limbs into its body and its ...",554,"{'chinese': '火红不倒翁', 'english': 'Darumaka', 'french': 'Da...",['Fire'],"{'HP': 70, 'Attack': 90, 'Defense': 45, 'Sp. Attack': 15,...",Zen Charm Pokémon,"{'next': [['555', 'use Ice Stone']], 'prev': None}","{'ability': [['Hustle', 'false'], ['Inner Focus', 'true']...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",553,0.430829


## Performing filters

### With vexpresso we can leverage duckdb for powerful queries

#### In order to filter on metadata, you must supply a filter dictionary to the filter method. The dictionary must have the following structure:

```python
{
    <field>: {
        <filter_method>: <value>
    },
    <field>: {
        <filter_method>: <value>
    },
}

```

#### Query to only get Pokemon with the name "Snorlax"

In [12]:
filter_condition = {
    "name.english": {
        "eq":"Snorlax"
    },
}
sleepy_pokemon.filter(filter_condition).to_pandas()

,description,id,name,type,base,species,evolution,profile,image,embeddings_description,modal_map_index,score
0,It has no interest in anything other than eati...,143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'fren...",[Normal],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp....",Sleeping Pokémon,"{'next': None, 'prev': ['446', 'high Friendshi...","{'ability': [['Immunity', 'false'], ['Thick Fa...",{'hires': 'https://raw.githubusercontent.com/P...,"[0.003054492874071002, -0.01002109982073307, -...",142,0.561686


#### Query to only get Pokemon with the name "Snorlax" or "Pikachu"

In [13]:
filter_condition = {
    "name.english": [
        {
            "eq":"Snorlax"
        },
        {
            "eq":"Pikachu"
        },
    ],
}
sleepy_pokemon.filter(filter_condition).to_pandas()

,description,id,name,type,base,species,evolution,profile,image,embeddings_description,modal_map_index,score
0,It has no interest in anything other than eati...,143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'fren...",[Normal],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp....",Sleeping Pokémon,"{'next': None, 'prev': ['446', 'high Friendshi...","{'ability': [['Immunity', 'false'], ['Thick Fa...",{'hires': 'https://raw.githubusercontent.com/P...,"[0.003054492874071002, -0.01002109982073307, -...",142,0.561686
1,"While sleeping, it generates electricity in th...",25,"{'chinese': '皮卡丘', 'english': 'Pikachu', 'fren...",[Electric],"{'HP': 35, 'Attack': 55, 'Defense': 40, 'Sp. A...",Mouse Pokémon,"{'next': [['26', 'use Thunder Stone']], 'prev'...","{'ability': [['Static', 'false'], ['Lightning ...",{'hires': 'https://raw.githubusercontent.com/P...,"[0.03698570281267166, -0.06595141440629959, -0...",24,0.474437


#### Query to only get Pokemon with the name "Snorlax" or "Pikachu" using a list

In [14]:
filter_condition = {
    "name.english": {
        "isin":["Snorlax", "Pikachu"]
    }
}
sleepy_pokemon.filter(filter_condition).to_pandas()

,description,id,name,type,base,species,evolution,profile,image,embeddings_description,modal_map_index,score
0,It has no interest in anything other than eati...,143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'fren...",[Normal],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp....",Sleeping Pokémon,"{'next': None, 'prev': ['446', 'high Friendshi...","{'ability': [['Immunity', 'false'], ['Thick Fa...",{'hires': 'https://raw.githubusercontent.com/P...,"[0.003054492874071002, -0.01002109982073307, -...",142,0.561686
1,"While sleeping, it generates electricity in th...",25,"{'chinese': '皮卡丘', 'english': 'Pikachu', 'fren...",[Electric],"{'HP': 35, 'Attack': 55, 'Defense': 40, 'Sp. A...",Mouse Pokémon,"{'next': [['26', 'use Thunder Stone']], 'prev'...","{'ability': [['Static', 'false'], ['Lightning ...",{'hires': 'https://raw.githubusercontent.com/P...,"[0.03698570281267166, -0.06595141440629959, -0...",24,0.474437


#### Query to only get Pokemon with type Normal and attack > 100

In [15]:
filter_condition = {
    "type": {
        "contains":"Normal"
    },
    "base.Attack": {
        "gt":100
    }
}
sleepy_pokemon.filter(filter_condition).to_pandas()

,description,id,name,type,base,species,evolution,profile,image,embeddings_description,modal_map_index,score
0,It has no interest in anything other than eati...,143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'fren...",[Normal],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp....",Sleeping Pokémon,"{'next': None, 'prev': ['446', 'high Friendshi...","{'ability': [['Immunity', 'false'], ['Thick Fa...",{'hires': 'https://raw.githubusercontent.com/P...,"[0.003054492874071002, -0.01002109982073307, -...",142,0.561686
1,It remains asleep from birth to death as a res...,775,"{'chinese': '树枕尾熊', 'english': 'Komala', 'fren...",[Normal],"{'HP': 65, 'Attack': 115, 'Defense': 65, 'Sp. ...",Drowsing Pokémon,"{'next': None, 'prev': None}","{'ability': [['Comatose', 'false']], 'egg': ['...",{'hires': 'https://raw.githubusercontent.com/P...,"[-0.01760929264128208, -0.03543251007795334, -...",774,0.421351


## Query + Filter chaining

#### For example, let's filter the above collection for Pokemon with HP >= 35 and Defense > 45

In [16]:
filter_conditions = {
    "base.HP": {"gte":35},
    "base.Defense": {"gt":45}
}
sleepy_pokemon = collection.query(query={"description":["Loves to sleep"]}, k=10).filter(filter_conditions=filter_conditions).execute()
sleepy_pokemon.show(5)

descriptionUtf8,idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",embeddings_descriptionPython,modal_map_indexInt64,scoreFloat64
It has no interest in anything other than eating. Even if...,143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'french': 'Ronfl...",['Normal'],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp. Attack': 6...",Sleeping Pokémon,"{'next': None, 'prev': ['446', 'high Friendship']}","{'ability': [['Immunity', 'false'], ['Thick Fat', 'false'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",105,0.561686
"It feels relaxed in tight, dark places and has been known...",722,"{'chinese': '木木枭', 'english': 'Rowlet', 'french': 'Brindi...","['Grass', 'Flying']","{'HP': 68, 'Attack': 55, 'Defense': 55, 'Sp. Attack': 50,...",Grass Quill Pokémon,"{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Reach', 'true'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",562,0.445208
"Although it’s tough, it can’t handle cold well. When nigh...",127,"{'chinese': '凯罗斯', 'english': 'Pinsir', 'french': 'Scarab...",['Bug'],"{'HP': 65, 'Attack': 125, 'Defense': 100, 'Sp. Attack': 5...",Stag Beetle Pokémon,"{'next': None, 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Mold Breaker', ...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",91,0.427454
It remains asleep from birth to death as a result of the ...,775,"{'chinese': '树枕尾熊', 'english': 'Komala', 'french': 'Bouma...",['Normal'],"{'HP': 65, 'Attack': 115, 'Defense': 65, 'Sp. Attack': 75...",Drowsing Pokémon,"{'next': None, 'prev': None}","{'ability': [['Comatose', 'false']], 'egg': ['Field'], 'g...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",604,0.421351
Delcatty sleeps anywhere it wants without keeping a perma...,301,"{'chinese': '优雅猫', 'english': 'Delcatty', 'french': 'Delc...",['Normal'],"{'HP': 70, 'Attack': 65, 'Defense': 65, 'Sp. Attack': 55,...",Prim Pokémon,"{'next': None, 'prev': ['300', 'use Moon Stone']}","{'ability': [['Cute Charm', 'false'], ['Normalize', 'fals...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,"<np.ndarrayshape=(768,)dtype=float64>",212,0.394259


## Grab fields from metadata

In [17]:
names, types, descriptions = sleepy_pokemon.select("name.english", "type", "description").to_dict().values()
for n, t, d in zip(names, types, descriptions):
    print(f"{n}: {t} -- {d}")

ValueError: DaftError::TypeError can not cast Utf8 to type: Boolean: Arrow types not castable

## Save collection

In [ ]:
sleepy_pokemon.save("./saved_filtered_pokemon")

## Load collection

In [ ]:
loaded_collection = vexpresso.create(
    directory_or_repo_id = "saved_filtered_pokemon",
    embedding_fn=HuggingFaceEmbeddings(),
    retriever=NumpyRetriever()
)

In [ ]:
loaded_collection.show(5)

## Uploading to HFhub

In [ ]:
# Automatically gets token from env variable
# HUGGINGFACEHUB_API_TOKEN = ...
# or you can pass in token directly via collection.save(token=...)
# repo_id = {HF_USERNAME}/{REPO_NAME}
repo_id = "shyamsn97/pokemon_vexpresso_test"
loaded_collection.save(repo_id, to_hub=True, private=True)

In [ ]:
# or upload with username + repo name
loaded_collection.save(hf_username = "shyamsn97", repo_name = "pokemon_vexpresso_test", to_hub=True)

## Load from HfHub

In [ ]:
# Automatically gets token from env variable
# HUGGINGFACEHUB_API_TOKEN = ...
# or you can pass in token directly via Collection.load(token=...)
# repo_id = {HF_USERNAME}/{REPO_NAME}
repo_id = "shyamsn97/pokemon_vexpresso_test"

collection = vexpresso.create(
    directory_or_repo_id = repo_id,
    embedding_fn=HuggingFaceEmbeddings(),
    retriever=NumpyRetriever()
)
collection.to_pandas()

In [ ]:
# or load with username + repo name
collection = vexpresso.create(
    hf_username = "shyamsn97",
    repo_name = "pokemon_vexpresso_test",
    embedding_fn=HuggingFaceEmbeddings(),
    retriever=NumpyRetriever()
)
collection.to_pandas()